![](../images/logos.jpg "MiCMOR, KIT Campus Alpin")

**[MiCMOR](https://micmor.kit.edu) [SummerSchool "Environmental Data Science: From Data Exploration to Deep Learning"](https://micmor.kit.edu/sites/default/files/MICMoR%20Summer%20School%202019%20Flyer.pdf)**  
IMK-IFU KIT Campus Alpin, Sept. 4 - 13 2019, Garmisch-Partenkirchen, Germany.

---

# Tidy data

As mentioned in the lecture, a tidy dataset is essential for alter analysis. This means, NoData shold be dealt with, columns be selected and other reorganization and cleanup be done.

Usually you do this be chaining a list of pandas commands to and write a cleanup notebook with all those steps.

Another, very clean, alternartive is to use the excellent `pyjanitor` package. It does the same thing, but much more concise and neat.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [11]:
# we load janitor after importin pandas 
import pandas as pd
import janitor

Let's first  bring in the data. It's a bird call dataset:

>Winger BM, Weeks BC, Farnsworth A, Jones AW, Hennen M, Willard DE (2019) Nocturnal flight-calling behaviour predicts vulnerability to artificial light in migratory birds. Proceedings of the Royal Society B 286(1900):20190364. [https://doi.org/10.1098/rspb.2019.0364](https://doi.org/10.1098/rspb.2019.0364).


In [12]:
raw_birds = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-04-30/raw/Chicago_collision_data.csv")
raw_call = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-04-30/raw/bird_call.csv", sep=" ")
raw_light = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-04-30/raw/Light_levels_dryad.csv")

As always, let's peek at the data

In [13]:
raw_birds.head()

,Genus,Species,Date,Locality
0,Ammodramus,nelsoni,1982-10-03,MP
1,Ammodramus,nelsoni,1984-05-21,CHI
2,Ammodramus,nelsoni,1984-05-25,MP
3,Ammodramus,nelsoni,1985-10-08,MP
4,Ammodramus,nelsoni,1986-09-10,MP


In [14]:
raw_call.head()

,Species,Family,Collisions,Flight,Call,Habitat,Stratum
0,Zonotrichia,albicollis,Passerellidae,10133,Yes,Forest,Lower
1,Junco,hyemalis,Passerellidae,6303,Yes,Edge,Lower
2,Melospiza,melodia,Passerellidae,5124,Yes,Edge,Lower
3,Melospiza,georgiana,Passerellidae,4910,Yes,Open,Lower
4,Seiurus,aurocapilla,Parulidae,4580,Yes,Forest,Lower


In [15]:
raw_light.head()

,Date,Light_Score
0,2000-03-06,3
1,2000-03-08,15
2,2000-03-10,3
3,2000-03-31,3
4,2000-04-02,17


## Let's clean with PyJanitor

Pyjanitor provides additional method calls to standard pandas dataframe objects. The clean_names() method is one example which removes whitespace and lowercases all column names.

In [16]:
clean_light = raw_light.clean_names()

In [17]:
clean_light.head()

,date,light_score
0,2000-03-06,3
1,2000-03-08,15
2,2000-03-10,3
3,2000-03-31,3
4,2000-04-02,17


Pyjanitor champions the cleaning process using the **call chaining approach**. We use this here to provide multiple column renaming. As our dataframes have inconsistent column names we rename the columns in the raw_call dataframe.

In [18]:
clean_call = (
    raw_call
    .rename_column("Species", "Genus") # rename 'Species' column to 'Genus'
    .rename_column("Family", "Species") # rename 'Family' columnto 'Species'
)

In [19]:
clean_call.head()

,Genus,Species,Collisions,Flight,Call,Habitat,Stratum
0,Zonotrichia,albicollis,Passerellidae,10133,Yes,Forest,Lower
1,Junco,hyemalis,Passerellidae,6303,Yes,Edge,Lower
2,Melospiza,melodia,Passerellidae,5124,Yes,Edge,Lower
3,Melospiza,georgiana,Passerellidae,4910,Yes,Open,Lower
4,Seiurus,aurocapilla,Parulidae,4580,Yes,Forest,Lower


We can chain as many standard pandas commands as we like, along with any pyjanitor specific methods.

In [20]:
clean_birds = (
    raw_birds
    .merge(clean_call, how='left') # merge the raw_birds dataframe with clean_raw dataframe
    .select_columns(["Genus", "Species", "Date", "Locality", "Collisions", "Call", "Habitat", "Stratum"]) # include list of cols
    .clean_names()
    .rename_column("collisions", "family") # rename 'collisions' column to 'family' in merged dataframe
    .rename_column("call", "flight_call")
    .dropna() # drop all rows which contain a NaN
)

In [21]:
clean_birds.head()

,genus,species,date,locality,family,flight_call,habitat,stratum
89,Passerculus,sandwichensis,1978-10-27,MP,Passerellidae,Yes,Open,Lower\t
90,Passerculus,sandwichensis,1979-10-23,MP,Passerellidae,Yes,Open,Lower\t
91,Passerculus,sandwichensis,1980-04-19,MP,Passerellidae,Yes,Open,Lower\t
92,Passerculus,sandwichensis,1981-09-23,MP,Passerellidae,Yes,Open,Lower\t
93,Passerculus,sandwichensis,1982-05-20,MP,Passerellidae,Yes,Open,Lower\t


Very handy 👍!